## Imports and Data Set Up

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [3]:
data_dir = "./allendata"

manifest_path = os.path.join(data_dir, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)
sessions = cache.get_session_table()

## Initial Session Filtering

- select the `brain_observatory_1.1`
- analyse the brain_observatory sessions
- narrow down based on unit_count and the available stimuli


In [4]:
brain_obs_sessions = sessions[(sessions.session_type == "brain_observatory_1.1")]

print(f"There are {len(brain_obs_sessions)} out of {len(sessions)} sessions to choose from.")

There are 32 out of 58 sessions to choose from.


In [21]:
brain_obs_sessions["unit_count"].describe()

count     32.000000
mean     682.562500
std      142.865589
min      444.000000
25%      564.750000
50%      654.500000
75%      824.250000
max      959.000000
Name: unit_count, dtype: float64

Let's only keep the upper half of sessions, those with at least 650 units. Let's also specify the regions we want to explore: VISp, VISl, VISal, VISpm, and VISam.

In [ ]:
uh_brain_obs_sessions = brain_obs_sessions[(brain_obs_sessions["unit_count"] >= 650)]

required_regions = {"VISp", "VISl", "VISal", "VISpm", "VISam"}
regions_uh_brain_obs_sessions = uh_brain_obs_sessions[
    uh_brain_obs_sessions["ecephys_structure_acronyms"].apply(
        lambda x: all(region in str(x) for region in required_regions)
    )
]

regions_uh_brain_obs_sessions

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
750332458,2019-10-03T00:00:00Z,726141251,brain_observatory_1.1,91.0,M,wt/wt,902,2216,6,"[grey, VISrl, nan, VISal, IntG, IGL, LGd, CA3,..."
750749662,2019-10-03T00:00:00Z,726162197,brain_observatory_1.1,92.0,M,wt/wt,761,2223,6,"[LP, DG, CA1, VISp, nan, LGd, CA3, VISrl, VPM,..."
755434585,2019-10-03T00:00:00Z,730760270,brain_observatory_1.1,100.0,M,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,650,2220,6,"[grey, VISrl, nan, MGv, MGd, TH, LGd, CA3, DG,..."
756029989,2019-10-03T00:00:00Z,734865738,brain_observatory_1.1,96.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,684,2214,6,"[TH, DG, CA3, CA1, VISl, nan, PO, Eth, LP, VIS..."


Great! We have narrowed it down to 5 sessions. Let's download the first session, id="719161530".

In [ ]:
session = cache.get_session_data(719161530)

In [53]:
table = session.get_stimulus_table()

table["stimulus_name"].unique()

array(['spontaneous', 'gabors', 'invalid_presentation', 'flashes',
       'drifting_gratings', 'natural_movie_three', 'natural_movie_one',
       'static_gratings', 'natural_scenes'], dtype=object)

### Check value counts for each stimulus

In [ ]:
print("719161530")
table["stimulus_name"].value_counts()

719161530


natural_movie_three     36000
natural_movie_one       18000
static_gratings          6000
natural_scenes           5950
gabors                   3596
drifting_gratings         628
flashes                   150
invalid_presentation       49
spontaneous                15
Name: stimulus_name, dtype: int64

For the stimuli we are interested in, static and drifting gratings we have 6000 and 628 measurements respectively. Let's check the value counts for one more session, id="750332458"

In [ ]:
id2 = 750332458
session2 = cache.get_session_data(id2)

In [62]:
print(id2)
table2 = session2.get_stimulus_table()
table2["stimulus_name"].value_counts()

750332458


natural_movie_three    36000
natural_movie_one      18000
static_gratings         6000
natural_scenes          5950
gabors                  3645
drifting_gratings        630
flashes                  150
spontaneous               15
Name: stimulus_name, dtype: int64

Again a similar value at 6000 and 630 for the gratings. Due to the fact that Let's move forward with session 750332458.

## Exploring Session 750332458